In [6]:
%load_ext micropython_magic
%reload_ext micropython_magic

The micropython_magic extension is already loaded. To reload it, use:
  %reload_ext micropython_magic
Current ip.InteractiveTB.mode='Minimal'


In [18]:
# %mpy -s {"/dev/cu.usbmodem1234561"}
%mpy -s {"/dev/cu.usbmodem2101"}

In [26]:
# %%micropython

import time
from config import config
from secrets import secrets
from machine import RTC
from mp_libs.network import Network
from mp_libs.protocols import espnow_protocol
from mp_libs.protocols import min_iot_protocol
from mp_libs.protocols.wifi_protocols import WifiProtocol
from mp_libs.time import ptp

TIMEOUT_MSEC = 5000

rtc = RTC()
wifi = Network.create_wifi()
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport._transport

wifi.connect()
miniot_network.connect()
wifi.ntp_time_sync()

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")
print(f"Channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")

while True:
    rxed_packets = []
    data_available = False

    # Receive data
    while not data_available:
        data_available = miniot_network.receive(rxed_packets)
        time.sleep_ms(100)

    # Parse rx'ed packets
    for packet in rxed_packets:
        if ptp.is_ptp_msg(packet.msg):
            ptp_type, payload = ptp.parse_msg(packet.msg)
        else:
            print(f"Rx'ed unexpected message: {packet}")
            continue

        # Perform PTP sync
        if ptp_type == ptp.PtpMsg.SYNC_REQ:
            ptp.sequence_master(
                miniot_network,
                lambda miniot_msg: miniot_msg.msg,
                TIMEOUT_MSEC,
                num_sync_cycles=payload)

            print(f"now: {rtc.datetime()}")
            print(f"RTC now: {rtc.now()}")


2000-01-01 00:00:05.5981 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:05.5986 INFO-wifi-protocols:Connecting wifi...
2000-01-01 00:00:10.10765 INFO-wifi-protocols:Wifi is connected: ('192.168.0.45', '255.255.255.0', '192.168.0.1', '192.168.0.1')
2000-01-01 00:00:10.10768 INFO-miniot:Connecting MinIoT...
2000-01-01 00:00:10.10770 INFO-espnow-protocol:Connecting espnow...
Initial RTC datetime: (2025, 3, 10, 0, 0, 13, 10, 72)
Initial RTC now: 794880790000885
My MAC: b"H'\xe2\xc7n\xfc"
EPN Peer: b'p\x04\x1d\xad|\xc0'
Channel: 9
EPN timeout: 0
now: (2025, 3, 10, 0, 0, 13, 12, 608716)
RTC now: 794880792609812
now: (2025, 3, 10, 0, 0, 13, 28, 838517)
RTC now: 794880808839556
now: (2025, 3, 10, 0, 0, 13, 44, 968987)
RTC now: 794880824969714
now: (2025, 3, 10, 0, 0, 14, 1, 231811)
RTC now: 794880841232486
now: (2025, 3, 10, 0, 0, 14, 17, 325742)
RTC now: 794880857326839
now: (2025, 3, 10, 0, 0, 14, 33, 444836)
RTC now: 794880873445803
now: (2025, 3, 10, 0, 0, 14, 49, 560864)
RTC n